# 🏦 Bank Statement Processor - Python Analysis

**Connected to Google Apps Script Project**
- Script ID: `1Y40DccCVEpn29uA3P0gvQpyWmINnM_9CVZ7YzLqSQPieFGUBd3s83oa9`
- Spreadsheet ID: `1XuvPyWNhB3WAOMHDO9wXkZ5AO36Cce13PH8PAojG9Eo`
- Last Sync: 2026-01-20

In [ ]:
# Install required packages
!pip install -q gspread google-auth pandas numpy

In [ ]:
# Authenticate with Google
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
import numpy as np

# Authorize gspread
creds, _ = default()
gc = gspread.authorize(creds)

print('✅ Authentication successful!')

In [ ]:
# Connect to spreadsheet
SPREADSHEET_ID = '1XuvPyWNhB3WAOMHDO9wXkZ5AO36Cce13PH8PAojG9Eo'

try:
    spreadsheet = gc.open_by_key(SPREADSHEET_ID)
    print(f'✅ Connected to: {spreadsheet.title}')
    print(f'📊 Sheets: {[ws.title for ws in spreadsheet.worksheets()]}')
except Exception as e:
    print(f'❌ Error: {e}')

In [ ]:
# Load transactions data
try:
    transactions_sheet = spreadsheet.worksheet('Transactions')
    data = transactions_sheet.get_all_values()
    
    if len(data) > 1:
        df = pd.DataFrame(data[1:], columns=data[0])
        print(f'✅ Loaded {len(df)} transactions')
        display(df.head(10))
    else:
        print('⚠️ No transaction data found')
except Exception as e:
    print(f'❌ Error loading transactions: {e}')

In [ ]:
# Basic Analysis
if 'df' in dir() and len(df) > 0:
    # Convert amount to numeric
    df['Amount'] = pd.to_numeric(df.iloc[:, 4], errors='coerce')
    
    # Summary statistics
    total_income = df[df['Amount'] > 0]['Amount'].sum()
    total_expenses = df[df['Amount'] < 0]['Amount'].sum()
    net_flow = total_income + total_expenses
    
    print('📊 Financial Summary')
    print('=' * 40)
    print(f'💰 Total Income:   ${total_income:,.2f}')
    print(f'💸 Total Expenses: ${abs(total_expenses):,.2f}')
    print(f'📈 Net Cash Flow:  ${net_flow:,.2f}')
    print('=' * 40)
else:
    print('⚠️ No data to analyze')

In [ ]:
# Category breakdown (if available)
if 'df' in dir() and len(df) > 0 and len(df.columns) > 3:
    category_col = df.columns[3]  # Assuming category is in column 4
    if category_col:
        category_summary = df.groupby(df.iloc[:, 3])['Amount'].agg(['sum', 'count']).round(2)
        category_summary.columns = ['Total Amount', 'Count']
        category_summary = category_summary.sort_values('Total Amount')
        print('📂 Category Breakdown:')
        display(category_summary)